In [1]:
import csv
import os

import biosppy.signals.ecg as ecg
import neurokit2 as nk
import tsfel
import emd
import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
import tensorflow as tf
import IPython.display as ipd
import matplotlib.pyplot as plt
import scipy as sp
from numpy.fft import rfft, irfft
from tqdm import tqdm
import pywt

import keras

from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential

from sklearn.impute import KNNImputer

from sklearn.feature_selection import SelectKBest, chi2
from scipy import stats
from statistics import pstdev,variance
from sklearn.preprocessing import normalize

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

from sklearn.model_selection import KFold
from sklearn.metrics import balanced_accuracy_score

from sktime.transformations.panel.rocket import MiniRocket, MiniRocketMultivariateVariable

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, make_scorer, f1_score
import lightgbm as lgb
import xgboost as xgb

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.impute import SimpleImputer




In [2]:
X_train = pd.read_csv("task2/X_train.csv").drop('id', axis=1)
y_train = pd.read_csv("task2/y_train.csv").drop('id', axis=1)
X_test = pd.read_csv("task2/X_test.csv").drop('id', axis=1)

In [30]:
dataset_x = np.array(X_train)
for i in range(1):
    signal = dataset_x[i]
    signal = signal[~np.isnan(signal)]
    signal_serie = pd.Series(signal)
    corr = signal_serie.autocorr(lag=2)
    print(type(signal))
    print(signal.shape)
    print(np.average(signal))

<class 'numpy.ndarray'>
(16322,)
9.16431809827227


### Feature Extraction

In [36]:
def feature_extraction(dataset_x):

    #compute some statistics of all the signals 
    autocor = []
    ptp = []
    med = []
    avg = []
    fft=[]

    n_rows = dataset_x.shape[0]
    n_cols = dataset_x.shape[1]

    dataset_x = np.array(dataset_x)

    for i in range(n_rows):
        signal = dataset_x[i]
        signal = signal[~np.isnan(signal)]

        signal_serie = pd.Series(signal)
        corr = signal_serie.autocorr(lag=2)
        autocor.append(corr)
        #average, range, median
        avg.append(np.average(signal))
        ptp.append(np.ptp(signal))
        med.append(np.median(signal))
        #top 20 fft frequencies
        f = np.fft.fft(signal)
        array = f[0:800]
        n = 20
        indices = array.argsort()[-n:][::-1]
        fft.append(indices)


    #Padding the data in case some library need, padding with the last value. 
    to_pad = n_cols
    new_seq = []
    for one_seq in dataset_x:
        one_seq = one_seq[~np.isnan(one_seq)]
        len_one_seq = len(one_seq)
        last_val = one_seq[-1]
        n = to_pad - len_one_seq
        to_concat = np.repeat(0, n)
        new_one_seq = np.concatenate([one_seq, to_concat])
        new_seq.append(new_one_seq)
    padded = np.stack(new_seq)
    padded.shape

    #templates_ts: equal step timestep, axis reference
    #templates: #heartbeat * timestep 
    #heart_rate_ts: heart_rate_ts, equal-step timestamp
    #heart_rate: heart rate at each time

    ts_list = []
    filtered_list=[]
    rpeaks_list=[]
    templates_ts_list=[]
    templates_list=[]
    heart_rate_ts_list=[]
    heart_rate_list=[]
    norm_average_heartbeat_list = [] #normalized average heartbeat of pacient
    for i in range(n_rows):
        ts, filtered, rpeaks, templates_ts, templates, heart_rate_ts, heart_rate = ecg.ecg(signal=padded[i], sampling_rate=300.0, show=False)
        ts_list.append(ts)
        filtered_list.append(filtered)
        rpeaks_list.append(rpeaks)
        templates_ts_list.append(templates_ts)
        templates_list.append(templates)
        heart_rate_ts_list.append(heart_rate_ts)
        heart_rate_list.append(heart_rate)
        norm_template = normalize(templates)
        norm_average_heartbeat_list.append(sum(norm_template)/len(norm_template))

    # Energy of the signal
    energy_list = []
    for i in range(len(norm_average_heartbeat_list)):
        energy_list.append(np.sum(norm_average_heartbeat_list[i] ** 2))   


    # Extarct all the peaks 
    P_list=[]
    Q_list=[]
    R_list=[]
    S_list=[]
    T_list=[]
    for i in range(len(norm_average_heartbeat_list)):
        patient_current = norm_average_heartbeat_list[i]
        # Find the peak
        index = np.where(patient_current==max(patient_current))
        R = index[0]
        # First-half
        first_half = patient_current[0:R[0]]
        index = np.where(patient_current==min(first_half[R[0]-30:R[0]]))
        Q = index[0]
        index = np.where(first_half[0:Q[0]]==max(first_half[0:Q[0]]))
        P = index[0]
        #Second half
        second_half = patient_current[R[0]+1:] 
        index = np.where(patient_current==min(second_half[0:30]))
        S = index[0]
        second_half = second_half[S[0]-R[0]+1:]
        index = np.where(patient_current==max(second_half))
        T = index[0] 
        P_list.append(P[0])
        Q_list.append(Q[0])
        R_list.append(R[0])
        S_list.append(S[0])
        T_list.append(T[0])

    # Intervals and Ratios of peaks
    PR_list = []
    QRS_list = []
    ST_list = []
    for i in range(len(P_list)):
        PR_list.append(R_list[i]-P_list[i])
        QRS_list.append(S_list[i]-Q_list[i])
        ST_list.append(T_list[i]-S_list[i])

    #statistics of the rpeaks
    rpeaks_mean_list=[]
    rpeaks_median_list=[]
    rpeaks_var_list = []
    for i in range(len(rpeaks_list)):
        rpeaks_mean_list.append(np.mean(rpeaks_list[i]))
        rpeaks_median_list.append(np.median(rpeaks_list[i]))
        rpeaks_var_list.append(np.var(rpeaks_list[i]))

    #statistics of heartrate and heartbeat
    max_A=[]
    min_A=[]
    mean_A=[]
    median_A=[]

    for i in range(len(norm_average_heartbeat_list)):
        patient_current = norm_average_heartbeat_list[i]
        max_A.append(max(patient_current))
        min_A.append(min(patient_current))
        mean_A.append(np.mean(patient_current))
        median_A.append(np.median(patient_current))


    hr_mean_list = []
    hr_median_list = []
    hr_var_list = []
    for i in range(len(heart_rate_list)):
            hr_mean_list.append(np.mean(heart_rate_list[i]))
            hr_median_list.append(np.median(heart_rate_list[i]))
            hr_var_list.append(np.var(heart_rate_list[i]))
    hb_mean_list = []
    hb_median_list = []
    hb_var_list = []
    for i in range(len(norm_average_heartbeat_list)):
            hb_mean_list.append(np.mean(norm_average_heartbeat_list[i]))
            hb_median_list.append(np.median(norm_average_heartbeat_list[i]))
            hb_var_list.append(np.var(norm_average_heartbeat_list[i]))



    #calculate the statistics of difference of heartrate
    hr_diff_mean_list=[]
    hr_diff_median_list=[]
    hr_diff_var_list=[]
    for i in range(len(heart_rate_list)):
        d =np.diff(heart_rate_list[i])
        hr_diff_mean_list.append(np.mean(d))
        hr_diff_median_list.append(np.median(d))
        hr_diff_var_list.append(np.var(d))    

    # calculate the statistics of difference heartbeats
    hb_diff_mean_list=[]
    hb_diff_median_list=[]
    hb_diff_var_list=[]
    for i in range(len(templates_list)):
        d =np.diff(templates_list[i])
        hb_diff_mean_list.append(np.mean(d))
        hb_diff_median_list.append(np.median(d))  
        hb_diff_var_list.append(np.var(d))

    cA_list=[]
    cD_list=[]

    for i in range(len(norm_average_heartbeat_list)):
        cA, cD = pywt.dwt(norm_average_heartbeat_list[i], 'db2', mode='periodic')
        cA_list.append(cA)
        cD_list.append(cD)


    np_autocor = np.array([autocor]).reshape(-1,1)
    np_ptp = np.array([ptp]).reshape(-1,1)
    np_med = np.array([med]).reshape(-1,1)
    np_avg = np.array([avg]).reshape(-1,1)
    np_fft = np.array(fft)
    np_energy =np.array(energy_list).reshape(-1,1)


    np_P = np.array(P_list).reshape(-1,1)
    np_Q = np.array(Q_list).reshape(-1,1)
    np_R = np.array(R_list).reshape(-1,1)
    np_S = np.array(S_list).reshape(-1,1)
    np_T = np.array(T_list).reshape(-1,1)

    np_PR = np.array(PR_list).reshape(-1,1)
    np_QRS = np.array(QRS_list).reshape(-1,1)
    np_ST = np.array(ST_list).reshape(-1,1)

    np_QRS_T= np.divide(QRS_list, T_list) 
    np_QRS_P= np.divide(QRS_list, P_list) 
    np_QRS_T=np.nan_to_num(np_QRS_T, nan=0.0,posinf=0.0, neginf=0.0).reshape(-1,1)
    np_QRS_P=np.nan_to_num(np_QRS_P, nan=0.0,posinf=0.0, neginf=0.0).reshape(-1,1)

    np_rpeaks_mean = np.array(rpeaks_mean_list).reshape(-1,1)
    np_rpeaks_median = np.array(rpeaks_median_list).reshape(-1,1)
    np_rpeaks_var = np.array(rpeaks_var_list).reshape(-1,1)

    max_A=np.array(max_A).reshape(-1,1)
    min_A=np.array(min_A).reshape(-1,1)
    mean_A=np.array(mean_A).reshape(-1,1)
    median_A=np.array(median_A).reshape(-1,1)

    np_hr_mean = np.array(hr_mean_list).reshape(-1,1)
    np_hr_median = np.array(hr_median_list).reshape(-1,1)
    np_hr_var = np.array(hr_var_list).reshape(-1,1)
    
    nans_hr = np_hr_mean.isna().sum()
    print("is there Nan:",nans_hr)

    np_hb_mean = np.array(hb_mean_list).reshape(-1,1)
    np_hb_median = np.array(hb_median_list).reshape(-1,1)
    np_hb_var = np.array(hb_var_list).reshape(-1,1)   

    np_hr_diff_mean = np.array(hr_diff_mean_list).reshape(-1,1)
    np_hr_diff_median = np.array(hr_diff_median_list).reshape(-1,1)
    np_hr_diff_var = np.array(hr_diff_var_list).reshape(-1,1)
    np_hb_diff_mean = np.array(hb_diff_mean_list).reshape(-1,1)
    np_hb_diff_median = np.array(hb_diff_median_list).reshape(-1,1)
    np_hb_diff_var = np.array(hb_diff_var_list).reshape(-1,1)

        

    feature_set = np.concatenate((    
        np_autocor,
        np_ptp,
        np_med,
        np_avg,
        np_fft,
        np_energy,
        
        np_P,
        np_Q,
        np_R,
        np_S,
        np_T,

        np_PR,
        np_QRS,
        np_ST,
        np_QRS_T,
        np_QRS_P,   

        np_rpeaks_mean,
        np_rpeaks_median,
        np_rpeaks_var,

        max_A-min_A,
        mean_A,
        median_A,

        np_hr_mean,
        np_hr_median,
        np_hr_var,
        np_hb_mean,
        np_hb_median,
        np_hb_var,   

        np_hr_diff_mean,
        np_hr_diff_median,
        np_hr_diff_var,
        np_hb_diff_mean,
        np_hb_diff_median,
        np_hb_diff_var,
        cA_list,
        cD_list   

    ),axis = 1)

    print(feature_set.shape)
    #rocket features
    feature_extractor_rocket = MiniRocket(random_state=1, num_kernels = 100)
    rocket_features = feature_extractor_rocket.fit_transform(padded.reshape(padded.shape[0],1,padded.shape[1]))

    #tsfel features
    cfg = tsfel.get_features_by_domain('temporal')  # Or 'spectral' or 'statistical'
    extracted_features = []
    for i in range(padded.shape[0]):
        features = tsfel.time_series_features_extractor(cfg, padded[i], fs=300,verbose=0)
        extracted_features.append(features)
    tsfel_features = pd.concat(extracted_features, ignore_index=True)

    feature_set = pd.DataFrame(feature_set)

    concatenated_features = pd.concat([feature_set,rocket_features,tsfel_features],axis=1)


    return concatenated_features


In [37]:
X_train_features = feature_extraction(X_train)
#X_test_features = feature_extraction(X_test)

In [ ]:
nans_by_column = X_train_features.isna().sum()

print("NaNs count by column:")
print(nans_by_column)

NaNs count by column:
0                            0
1                            0
2                            0
3                            0
4                            0
                            ..
0_Positive turning points    0
0_Signal distance            0
0_Slope                      0
0_Sum absolute diff          0
0_Zero crossing rate         0
Length: 333, dtype: int64


In [34]:
 X_test_features.isna().sum().sum()

12

In [35]:
# Assuming 'series' is your pandas Series

# Set option to display maximum rows
pd.set_option('display.max_rows', None)

# Print the entire Series
print(nans_by_column)

# Reset the option back to default
pd.reset_option('display.max_rows')


0                            0
1                            0
2                            0
3                            0
4                            0
5                            0
6                            0
7                            0
8                            0
9                            0
10                           0
11                           0
12                           0
13                           0
14                           0
15                           0
16                           0
17                           0
18                           0
19                           0
20                           0
21                           0
22                           0
23                           0
24                           0
25                           0
26                           0
27                           0
28                           0
29                           0
30                           0
31                           0
32      

In [107]:
X_train_features.columns = X_train_features.columns.astype(str)
X_test_features.columns = X_test_features.columns.astype(str)


imputer = SimpleImputer(strategy='median')
X_train_features = pd.DataFrame(imputer.fit_transform(X_train_features), columns=X_train_features.columns)
X_test_features = pd.DataFrame(imputer.fit_transform(X_test_features), columns=X_train_features.columns)
y_train = pd.read_csv("task2/y_train.csv").drop('id', axis=1)




c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\impute\_base.py:555: UserWarning: Skipping features without any observed values: ['1' '2' '3']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


ValueError: Shape of passed values is (3411, 330), indices imply (3411, 333)

In [ ]:
select_k_best = SelectKBest(f_classif, k=300)  # Adjust 'k' as needed
X_new = select_k_best.fit_transform(X_train_features, y_train)
x_train, x_test, y_train_, y_test_ = train_test_split(X_new, y_train, test_size=0.20, random_state=42)

clf = GradientBoostingClassifier(learning_rate=0.05, n_estimators=500, max_depth=7, 
                                 min_samples_split=60, min_samples_leaf=9, subsample=1.0,
                                 max_features=50, random_state=0)


print ('Training')
eval_set = [(x_test, y_test_)]
clf.fit(x_train, y_train_)
    
print ('Predicting')
predicted_labels = clf.predict(x_test)

print ('Scoring')
score = f1_score(y_test_, predicted_labels, average='micro')

print('>%.3f' % score)

print ('Scoring')
confusion_matrix(y_test_, predicted_labels)

c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [1 2] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training
Predicting
Scoring
>0.813
Scoring


array([[557,   0,  37,   1],
       [  5,  61,  18,   2],
       [ 92,  12, 194,   4],
       [ 13,   1,   6,  21]], dtype=int64)

In [ ]:
X_test_new = select_k_best.transform(X_test_features)
clf = GradientBoostingClassifier(learning_rate=0.05, n_estimators=500, max_depth=7, 
                                 min_samples_split=60, min_samples_leaf=9, subsample=1.0,
                                 max_features=50, random_state=0)
clf.fit(X_new, y_train)


predicted_labels = clf.predict(X_test_new)


c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\ensemble\_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Predicting


c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


ValueError: X has 333 features, but GradientBoostingClassifier is expecting 300 features as input.

In [ ]:
X_test_features

In [110]:
X_test_new = select_k_best.transform(X_test_features)
predicted_labels = clf.predict(X_test_new)


ValueError: Input X contains NaN.
SelectKBest does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
X_test_new.shape

(5117, 300)

In [ ]:
f = open("FirstMLModel.csv", "w")
f.write("id,y\n")
for i,x in enumerate(predicted_labels):
    f.write("{},{}\n".format(i,predicted_labels[i]))
f.close()

In [ ]:
len(predicted_labels)

5117

In [ ]:
X_test_features.shape

(5117, 300)

In [36]:



y_train = pd.read_csv("task2/y_train.csv").drop('id', axis=1)

# Apply SelectKBest to extract top features
select_k_best = SelectKBest(f_classif, k=200)  # Adjust 'k' as needed
X_new = select_k_best.fit_transform(concatenated_features, y_train)
X_train, X_test, y_train, y_test = train_test_split(X_new, y_train, test_size=0.25, random_state=1)

models_and_parameters = {
    'RandomForest': (RandomForestClassifier(), 
                     {'n_estimators': [50, 100], 'max_depth': [5, 10]}),
    'LightGBM': (lgb.LGBMClassifier(), 
                 {'learning_rate': [0.01, 0.1], 'n_estimators': [50, 100]}),
    'XGBoost': (xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss'), 
                {'learning_rate': [0.01, 0.1], 'n_estimators': [50, 100]})
}

# Define the F1 scorer
f1_scorer = make_scorer(f1_score, average='weighted')

# Iterate over models and perform grid search
for model_name, (model, param_grid) in models_and_parameters.items():
    clf = GridSearchCV(model, param_grid, cv=5, scoring=f1_scorer)
    clf.fit(X_train, y_train)
    print(f"Best parameters for {model_name}: {clf.best_params_}")
    predictions = clf.predict(X_test)
    print(f"F1 Score for {model_name}: {f1_score(y_test, predictions, average='weighted')}")
    print(f"Classification Report for {model_name}:\n{classification_report(y_test, predictions)}")



c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [  1   2 946] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:113: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\base.py:1151: Dat

Best parameters for RandomForest: {'max_depth': 10, 'n_estimators': 100}
F1 Score for RandomForest: 0.6984935593305222
Classification Report for RandomForest:
              precision    recall  f1-score   support

           0       0.73      0.94      0.82       733
           1       0.73      0.32      0.44       111
           2       0.71      0.48      0.57       385
           3       0.70      0.31      0.43        51

    accuracy                           0.72      1280
   macro avg       0.72      0.51      0.57      1280
weighted avg       0.72      0.72      0.70      1280

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.006254 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 49835
[LightGBM] [Info] Number of data points in the train set: 3069, number of used features: 200
[LightGBM] [Info] Start training from score -0.513218
[LightGBM] [Info] Start training from score -2.445611
[LightGB

c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007002 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 49814
[LightGBM] [Info] Number of data points in the train set: 3069, number of used features: 200
[LightGBM] [Info] Start training from score -0.513218
[LightGBM] [Info] Start training from score -2.445611
[LightGBM] [Info] Start training from score -1.259465
[LightGBM] [Info] Start training from score -3.475230
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


...

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007535 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 49827
[LightGBM] [Info] Number of data points in the train set: 3070, number of used features: 200
[LightGBM] [Info] Start training from score -0.513000
[LightGBM] [Info] Start training from score -2.449703
[LightGBM] [Info] Start training from score -1.258643
[LightGBM] [Info] Start training from score -3.475556
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007258 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 49817
[LightGBM] [Info] Number of data points in the train set: 3070, number of used features: 200
[LightGBM] [Info] Start training from score -0.513000
[LightGBM] [Info] Start training from score -2.445937
[LightGBM] [Info] Start training from score -1.259791
[LightGBM] [Info] Start training from score -3.475556
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\zheng\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008028 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 49834
[LightGBM] [Info] Number of data points in the train set: 3837, number of used features: 200
[LightGBM] [Info] Start training from score -0.513087
[LightGBM] [Info] Start training from score -2.447311
[LightGBM] [Info] Start training from score -1.259431
[LightGBM] [Info] Start training from score -3.473323
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Best parameters for LightGBM: {'learning_rate': 0.1, 'n_estimators': 100}
F1 Score for LightGBM: 0.7670893681048637
Classification Report for LightGBM:
              precision    recall  f1-score   support

           0       0.80      0.94      0.86       733
           1       0.77      0.50      0.60       111
           2       0.75      0.58      0.65       385
           3       0.65      0.55      0.60       

### Neorokit features

In [ ]:
neurokit_features = nk.ecg_analyze(X_train, sampling_rate=300)
neurokit_features

ValueError: NeuroKit error: ecg_intervalrelated(): Wrong input,we couldn't extract heart rate. Please make sureyour DataFrame contains an `ECG_Rate` column.

In [ ]:
def process_ecg(signal, sampling_rate=300):
    signals, info = nk.ecg_process(signal, sampling_rate=sampling_rate)
    features = nk.ecg_intervalrelated(signals, sampling_rate=sampling_rate)
    return features


neurokit_features_df = pd.DataFrame()
# Iterate over each row in the dataframe
for i in range(padded.shape[0]):
    row = padded[i]
    # Process each row and extract features
    features = process_ecg(row)    
    # Append the features to the features_df
    neurokit_features_df = pd.concat([neurokit_features_df, features], ignore_index=True)





c:\Users\zheng\anaconda3\Lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
c:\Users\zheng\anaconda3\Lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
c:\Users\zheng\anaconda3\Lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` ar

LinAlgError: SVD did not converge in Linear Least Squares

In [ ]:
data = nk.data("bio_resting_5min_100hz")

In [ ]:
data

,ECG,PPG,RSP
0,0.003766,-0.102539,0.494652
1,-0.017466,-0.103760,0.502483
2,-0.015679,-0.107422,0.511102
3,-0.001598,-0.110855,0.518791
4,0.002483,-0.112610,0.528669
...,...,...,...
29995,-0.045986,-0.135498,0.981111
29996,-0.107841,-0.155334,0.978866
29997,-0.058274,-0.173721,0.976366
29998,-0.056472,-0.192108,0.972202


In [ ]:
df, info = nk.ecg_process(data["ECG"], sampling_rate=100)

In [ ]:
nk.ecg_intervalrelated(df, sampling_rate=100)

,ECG_Rate_Mean,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,HRV_SDANN5,HRV_SDNNI5,HRV_RMSSD,...,HRV_LZC,HRV_DFA_alpha2,HRV_MFDFA_alpha2_Width,HRV_MFDFA_alpha2_Peak,HRV_MFDFA_alpha2_Mean,HRV_MFDFA_alpha2_Max,HRV_MFDFA_alpha2_Delta,HRV_MFDFA_alpha2_Asymmetry,HRV_MFDFA_alpha2_Fluctuation,HRV_MFDFA_alpha2_Increment
0,86.392105,[[694.7563805104409]],[[49.03604322104105]],[[7.277184736458611]],[[48.83361026572851]],[[nan]],[[nan]],[[nan]],[[nan]],[[38.83776632381496]],...,[[0.8731238852455482]],[[0.74458349456187]],[[0.12025402871845392]],[[0.6260707670636847]],[[0.6861977814229117]],[[1.0034826024405372]],[[-0.09163628837599647]],[[0.0]],[[2.6738753815679935e-05]],[[0.0018592659897405457]]
